In [9]:
import os
import pandas as pd
import natsort
data_dir =  '../cropped/cropped'
data_per = os.listdir(data_dir)

In [10]:
def max_frame(data_per):
    max_frames = 0
    for person in data_per:
        if person == '.DS_Store':
            continue
        words_dir = f'{data_dir}/{person}/words'
        Id = os.listdir(words_dir)
        for id in Id :
            id_dir = f'{data_dir}/{person}/words/{id}'
            num = os.listdir(id_dir)
            for n  in num:
                #print("n is", n)
                words_dir_2 = f'{data_dir}/{person}/words/{id}/{n}'
                frames = os.listdir(words_dir_2)
                #print(len(frames))
                max_frames = max(len(frames), max_frames)
    return max_frames

In [26]:
def concat_x_data(x_data, x_data_):
    if x_data_.shape[0] == max_frame(data_per):
        return np.concatenate([x_data, x_data_], axis = 1) 
    else: 
        pad = max_frame(data_per) - x_data_.shape[1]
        print("pad for ",pad)
        zero_pad = np.zeros((1, pad, 64, 64, 3))
        print(zero_pad.shape)
#         print(x_data_.shape)
        data = np.concatenate([x_data_, zero_pad], axis = 1) #padding
        return data

In [34]:
from tensorflow.keras.preprocessing import image
import numpy as np

for person in natsort.natsorted(data_per):
    if person == '.DS_Store':
        continue
    #phrases_dir = f'{data_dir}/{person}/phrases'
    words_dir = f'{data_dir}/{person}/words'
    Id = os.listdir(words_dir)
    for sentence in natsort.natsorted(Id) :
        id_dir = f'{data_dir}/{person}/words/{sentence}'
        num = os.listdir(id_dir)
        for n  in natsort.natsorted(num):
            print("person: {}, sentence: {}, n: {} ".format(person, sentence, n))
            words_dir_2 = f'{data_dir}/{person}/words/{sentence}/{n}'
            frames = os.listdir(words_dir_2)
            x_data_ = np.empty(4)
            x_data_aug = np.empty(4)
            for frame in natsort.natsorted(frames):
                img_path = f'{words_dir_2}/{frame}'
                
                # flatten 제거한 frame processing
                img = image.load_img(img_path, target_size=(64, 64))
                img_tensor = image.img_to_array(img) #tensor로 type 변경
                img_tensor /= 255. # scaling 
                img_tensor = np.expand_dims(img_tensor, axis = 0) # (1, 64, 64, 3) 차원
                
                # frame 별 묶기
                if frame == 'color_001.jpg':
                    x_data_ = img_tensor
                    x_data_aug = DataAugmentation().generate(img)
                else:
                    x_data_ = np.concatenate([x_data_, img_tensor], axis = 0)
                    img_augmetned = DataAugmentation.generate(img)
                    x_data_aug = np.concatenate([x_data_aug, img_augmetned], axis=0)
                
                print(x_data_.shape)
                
            # 이미지 전체 통합시키기
            x_data_ = np.expand_dims(x_data_, axis = 0)  #[1, frame, 64, 64, 3]
            x_data_aug = np.expand_dims(x_data_aug, axis=0)
            if (n == '01' and sentence == '01' and person == 'F01'):
                x_data = concat_x_data(x_data, x_data_)
                y_data = np.array([int(sentence)])
                data_aug = concat_x_data(x_data, x_data_aug)
                x_data = np.concatenate([x_data, data_aug], axis = 0)
                y_data = np.concatenate([y_data, [int(sentence)]])
            else:
                if x_data_.shape[0] == max_frame(data_per):
                    x_data = concat_x_data(x_data, x_data_)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                    x_data = concat_x_data(x_data, x_data_aug)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                else: 
                    data = concat_x_data(x_data, x_data_)
                    x_data = np.concatenate([x_data, data], axis = 0)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                    data_aug = concat_x_data(x_data, x_data_aug)
                    x_data = np.concatenate([x_data, data_aug], axis = 0)
                    y_data = np.concatenate([y_data, [int(sentence)]])
            print(x_data.shape)
            print("y.shape: {}" .format(y_data.shape))

person: F01, sentence: 01, n: 01 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
pad for  10
(1, 10, 64, 64, 3)
pad for  10
(1, 10, 64, 64, 3)
(2, 20, 64, 64, 3)
y.shape: (2,)
person: F01, sentence: 01, n: 02 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
pad for  13
(1, 13, 64, 64, 3)
pad for  13
(1, 13, 64, 64, 3)
(4, 20, 64, 64, 3)
y.shape: (4,)
person: F01, sentence: 01, n: 03 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
pad for  10
(1, 10, 64, 64, 3)
pad for  10
(1, 10, 64, 64, 3)
(6, 20, 64, 64, 3)
y.shape: (6,)
person: F01, sentence: 01, n: 04 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
pad for  11
(1, 11, 64, 64, 3)
pad fo

pad for  14
(1, 14, 64, 64, 3)
(66, 20, 64, 64, 3)
y.shape: (66,)
person: F02, sentence: 01, n: 04 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
pad for  14
(1, 14, 64, 64, 3)
pad for  14
(1, 14, 64, 64, 3)
(68, 20, 64, 64, 3)
y.shape: (68,)
person: F02, sentence: 01, n: 05 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
pad for  14
(1, 14, 64, 64, 3)
pad for  14
(1, 14, 64, 64, 3)
(70, 20, 64, 64, 3)
y.shape: (70,)
person: F02, sentence: 01, n: 06 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
pad for  14
(1, 14, 64, 64, 3)
pad for  14
(1, 14, 64, 64, 3)
(72, 20, 64, 64, 3)
y.shape: (72,)
person: F02, sentence: 01, n: 07 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
pad for  13
(1, 13, 64, 64, 3)
pad for  13
(1, 13, 64, 64, 3)
(74, 20, 64, 64, 3)
y.shape: (74,)
person: F02, sentence: 01, n: 08 
(

(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 64, 3)
(15, 64, 64, 3)
(16, 64, 64, 3)
pad for  4
(1, 4, 64, 64, 3)
pad for  4
(1, 4, 64, 64, 3)
(128, 20, 64, 64, 3)
y.shape: (128,)
person: F04, sentence: 01, n: 05 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 64, 3)
(15, 64, 64, 3)
(16, 64, 64, 3)
(17, 64, 64, 3)
(18, 64, 64, 3)
(19, 64, 64, 3)
pad for  1
(1, 1, 64, 64, 3)
pad for  1
(1, 1, 64, 64, 3)
(130, 20, 64, 64, 3)
y.shape: (130,)
person: F04, sentence: 01, n: 06 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 64, 3)
(15, 64, 64, 3)
(16, 64, 64, 3)
(17, 64, 64, 3)
(18, 64, 64, 3)
(19, 64, 64, 3)
pad for  1
(1, 1, 64, 64, 3)
pad for  1
(1, 1, 64, 64, 3)
(132, 20, 64, 

(170, 20, 64, 64, 3)
y.shape: (170,)
person: F04, sentence: 03, n: 06 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 64, 3)
(15, 64, 64, 3)
pad for  5
(1, 5, 64, 64, 3)
pad for  5
(1, 5, 64, 64, 3)
(172, 20, 64, 64, 3)
y.shape: (172,)
person: F04, sentence: 03, n: 07 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 64, 3)
(15, 64, 64, 3)
pad for  5
(1, 5, 64, 64, 3)
pad for  5
(1, 5, 64, 64, 3)
(174, 20, 64, 64, 3)
y.shape: (174,)
person: F04, sentence: 03, n: 08 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 6

pad for  7
(1, 7, 64, 64, 3)
(222, 20, 64, 64, 3)
y.shape: (222,)
person: F05, sentence: 03, n: 02 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 64, 3)
(15, 64, 64, 3)
(16, 64, 64, 3)
(17, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(224, 20, 64, 64, 3)
y.shape: (224,)
person: F05, sentence: 03, n: 03 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
pad for  7
(1, 7, 64, 64, 3)
pad for  7
(1, 7, 64, 64, 3)
(226, 20, 64, 64, 3)
y.shape: (226,)
person: F05, sentence: 03, n: 04 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 

pad for  9
(1, 9, 64, 64, 3)
(272, 20, 64, 64, 3)
y.shape: (272,)
person: F06, sentence: 02, n: 07 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
pad for  9
(1, 9, 64, 64, 3)
pad for  9
(1, 9, 64, 64, 3)
(274, 20, 64, 64, 3)
y.shape: (274,)
person: F06, sentence: 02, n: 08 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
pad for  9
(1, 9, 64, 64, 3)
pad for  9
(1, 9, 64, 64, 3)
(276, 20, 64, 64, 3)
y.shape: (276,)
person: F06, sentence: 02, n: 09 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
pad for  10
(1, 10, 64, 64, 3)
pad for  10
(1, 10, 64, 64, 3)
(278, 20, 64, 64, 3)
y.shape: (278,)
person: F06, sentence: 02, n: 10 
(1, 64, 64, 3)
(2, 64, 6

In [37]:
print(y_data[:100])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [23]:
# ref: https://m.blog.naver.com/PostView.nhn?blogId=isu112600&logNo=221582003889&proxyReferer=https:%2F%2Fwww.google.com%2F
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataAugmentation:
    @classmethod
    def generate(self, img):
        img_array = image.array_to_img(img)
        samples = np.expand_dims(img_array, axis=0)
        
        # generatior 생성
        datagen = ImageDataGenerator(
            width_shift_range=[-10,10], 
            height_shift_range=0.5, 
            rotation_range=45)
        
        it = datagen.flow(samples, batch_size=1)
        augmented = it.next()
        img_augmented = augmented[0]
        img_augmented /= 255. 
        img_augmented = np.expand_dims(img_augmented, 0)

        return img_augmented